<a href="https://colab.research.google.com/github/kodenshacho/SimpleVolumeRendering/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
std::queue<std::vector<unsigned char*>> textureQueue;
std::mutex queueMutex;
std::condition_variable queueCondVar;
bool done = false;

void DataLoadThread(const std::string& folderPath) {
    std::vector<unsigned char*> vtImages = DataLoad(folderPath);
    {
        std::lock_guard<std::mutex> lock(queueMutex);
        textureQueue.push(vtImages);
    }
    queueCondVar.notify_one();
}

void setupTexture(std::vector<unsigned char*> images) {
    for (size_t i = 0; i < images.size(); ++i) {
        glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, 128, 128, 0, GL_RGBA, GL_UNSIGNED_BYTE, images[i]);
    }
}
void RenderThread() {
    while (!done) {
        std::unique_lock<std::mutex> lock(queueMutex);
        queueCondVar.wait(lock, [] { return !textureQueue.empty() || done; });

        while (!textureQueue.empty()) {
            std::vector<unsigned char*> images = textureQueue.front();
            textureQueue.pop();
            lock.unlock();

            // OpenGL コンテキストが有効化されていることを確認
            // wglMakeCurrent(hdc, hglrc); // 例: Windowsの場合

            setupTexture(images);

            // OpenGL コンテキストを解放する場合がある
            // wglMakeCurrent(NULL, NULL); // 例: Windowsの場合

            lock.lock();
        }
    }
}



int main() {
    std::string folderPath = "image_path/folder_name";

    // OpenGL コンテキストの初期化
    // HDC hdc = ...;
    // HGLRC hglrc = wglCreateContext(hdc);
    // wglMakeCurrent(hdc, hglrc);

    std::thread renderThread(RenderThread);
    std::thread dataLoadThread(DataLoadThread, folderPath);

    // データロードスレッドが終了するのを待つ
    dataLoadThread.join();

    // レンダースレッドに終了を通知
    {
        std::lock_guard<std::mutex> lock(queueMutex);
        done = true;
    }
    queueCondVar.notify_one();
    renderThread.join();

    // OpenGL コンテキストのクリーンアップ
    // wglMakeCurrent(NULL, NULL);
    // wglDeleteContext(hglrc);

    return 0;
}



In [ ]:
#include <vector>
#include <thread>
#include <mutex>
#include <queue>
#include <iostream>
#include <string>
#include <GLES2/gl2.h>

using namespace std;

mutex mtx;
queue<vector<unsigned char*>> dataQueue;

vector<unsigned char*> DataLoad(const string& folder_path) {
    vector<unsigned char*> vtImages;
    return vtImages;
}

void setupTexture(vector<unsigned char*>& images) {
    for(size_t i = 0; i < images.size(); i++) {
        glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, 128, 128, 0, GL_RGBA, GL_UNSIGNED_BYTE, images[i]);
    }
}

void Render2D() {
    while (true) {
        unique_lock<mutex> lock(mtx);
        if (!dataQueue.empty()) {
            vector<unsigned char*> images = dataQueue.front();
            dataQueue.pop();
            lock.unlock();
            setupTexture(images);
        } else {
            lock.unlock();
            this_thread::sleep_for(chrono::milliseconds(10));
        }
    }
}

void LoadAndSendData(const string& folder_path) {
    vector<unsigned char*> images = DataLoad(folder_path);
    unique_lock<mutex> lock(mtx);
    dataQueue.push(images);
}

int main() {
    string folder_path = "image_path/folder_name";

    thread renderThread(Render2D);

    thread dataThread(LoadAndSendData, folder_path);

    renderThread.join();
    dataThread.join();

    return 0;
}


In [ ]:
#include <iostream>
#include <vector>
#include <queue>
#include <thread>
#include <mutex>
#include <condition_variable>
#include <GL/glew.h>
#include <GLFW/glfw3.h>

// Image data type
using ImageData = std::vector<unsigned char>;

// Thread-safe queue
template<typename T>
class ThreadSafeQueue {
public:
    void push(const T& value) {
        std::lock_guard<std::mutex> lock(mtx);
        q.push(value);
        cv.notify_one();
    }

    bool pop(T& value) {
        std::unique_lock<std::mutex> lock(mtx);
        cv.wait(lock, [this] { return !q.empty(); });
        value = q.front();
        q.pop();
        return true;
    }

private:
    std::queue<T> q;
    std::mutex mtx;
    std::condition_variable cv;
};

ThreadSafeQueue<ImageData> imageQueue;
bool running = true;

void Render2D() {
    while (running) {
        ImageData image;
        if (imageQueue.pop(image)) {
            // Bind your texture and use glTexImage2D to upload data
            GLuint texture;
            glGenTextures(1, &texture);
            glBindTexture(GL_TEXTURE_2D, texture);
            glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, 128, 128, 0, GL_RGBA, GL_UNSIGNED_BYTE, image.data());
            glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR);
            glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR);
            // Now the texture is set up and ready to use in rendering
        }
    }
}

std::vector<ImageData> DataLoad(const std::string& folder_path) {
    std::vector<ImageData> vtImages;
    // Load images from folder and push into vtImages
    // For the example, we'll just create dummy data
    for (int i = 0; i < 5; ++i) {
        ImageData img(128 * 128 * 4, static_cast<unsigned char>(i)); // Dummy image data
        vtImages.push_back(img);
    }
    return vtImages;
}

void setupTexture(const std::vector<ImageData>& images) {
    for (const auto& image : images) {
        imageQueue.push(image);
    }
}

int main() {
    // Initialize GLFW and GLEW, create a window and OpenGL context
    if (!glfwInit()) {
        std::cerr << "Failed to initialize GLFW" << std::endl;
        return -1;
    }

    GLFWwindow* window = glfwCreateWindow(640, 480, "OpenGL ES Example", nullptr, nullptr);
    if (!window) {
        std::cerr << "Failed to create GLFW window" << std::endl;
        glfwTerminate();
        return -1;
    }

    glfwMakeContextCurrent(window);
    glewExperimental = GL_TRUE;
    if (glewInit() != GLEW_OK) {
        std::cerr << "Failed to initialize GLEW" << std::endl;
        return -1;
    }

    std::thread renderThread(Render2D);

    // Load and setup textures
    std::vector<ImageData> images = DataLoad("image_path/folder_name");
    setupTexture(images);

    // Main loop
    while (!glfwWindowShouldClose(window)) {
        glClear(GL_COLOR_BUFFER_BIT);

        // Render your scene

        glfwSwapBuffers(window);
        glfwPollEvents();
    }

    running = false;
    renderThread.join();

    glfwDestroyWindow(window);
    glfwTerminate();
    return 0;
}


In [ ]:
#include <GL/glew.h>
#include <GLFW/glfw3.h>
#include <vector>
#include <queue>
#include <mutex>
#include <condition_variable>
#include <thread>
#include <iostream>

std::queue<std::vector<unsigned char*>> textureQueue;
std::mutex queueMutex;
std::condition_variable conditionVar;
bool done = false;

void Render2D();
std::vector<unsigned char*> DataLoad(const std::string& folder_path);
void setupTexture(const std::vector<unsigned char*>& images);
void LoadTexturesThread(const std::string& folder_path);
void ProcessTextures();

int main() {
    // Initialize GLFW
    if (!glfwInit()) {
        std::cerr << "Failed to initialize GLFW" << std::endl;
        return -1;
    }

    // Create a GLFW window
    GLFWwindow* window = glfwCreateWindow(800, 600, "OpenGL Example", nullptr, nullptr);
    if (!window) {
        std::cerr << "Failed to create GLFW window" << std::endl;
        glfwTerminate();
        return -1;
    }

    // Make the OpenGL context current
    glfwMakeContextCurrent(window);

    // Initialize GLEW
    GLenum err = glewInit();
    if (err != GLEW_OK) {
        std::cerr << "Failed to initialize GLEW: " << glewGetErrorString(err) << std::endl;
        return -1;
    }

    // Start the texture loading thread
    std::thread textureThread(LoadTexturesThread, "image_path/folder_name");

    while (!glfwWindowShouldClose(window)) {
        // Render here
        Render2D();

        // Swap buffers
        glfwSwapBuffers(window);

        // Poll for and process events
        glfwPollEvents();
    }

    // Clean up
    done = true;
    conditionVar.notify_all();
    textureThread.join();
    glfwDestroyWindow(window);
    glfwTerminate();
    return 0;
}

void Render2D() {
    // Process textures from the queue
    ProcessTextures();
    // Add rendering code here
}

std::vector<unsigned char*> DataLoad(const std::string& folder_path) {
    // Simulate loading images from folder
    std::vector<unsigned char*> vtImages;
    // Load your images into vtImages
    // For demonstration, we'll push dummy data
    for (int i = 0; i < 5; ++i) {
        vtImages.push_back(new unsigned char[128 * 128 * 4]); // RGBA
    }
    return vtImages;
}

void setupTexture(const std::vector<unsigned char*>& images) {
    for (size_t i = 0; i < images.size(); ++i) {
        GLuint textureID;
        glGenTextures(1, &textureID);
        glBindTexture(GL_TEXTURE_2D, textureID);
        glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, 128, 128, 0, GL_RGBA, GL_UNSIGNED_BYTE, images[i]);
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR);
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR);
        glBindTexture(GL_TEXTURE_2D, 0);
    }
}

void LoadTexturesThread(const std::string& folder_path) {
    while (!done) {
        std::vector<unsigned char*> images = DataLoad(folder_path);

        std::unique_lock<std::mutex> lock(queueMutex);
        textureQueue.push(images);
        conditionVar.notify_all();

        // Simulate load delay
        std::this_thread::sleep_for(std::chrono::seconds(1));
    }
}

void ProcessTextures() {
    std::unique_lock<std::mutex> lock(queueMutex);
    conditionVar.wait(lock, []{ return !textureQueue.empty() || done; });

    while (!textureQueue.empty()) {
        auto images = textureQueue.front();
        textureQueue.pop();
        lock.unlock();

        setupTexture(images);

        for (auto img : images) {
            delete[] img;
        }

        lock.lock();
    }
}


In [ ]:
#include <iostream>
#include <thread>
#include <queue>
#include <vector>
#include <mutex>
#include <condition_variable>
#include <GL/glew.h>
#include <GLFW/glfw3.h>

// 画像データを保持する構造体
struct ImageData {
    unsigned char* data;
    int width;
    int height;
};

// スレッド間で共有するキューと同期用の変数
std::queue<ImageData> imageQueue;
std::mutex queueMutex;
std::condition_variable queueCondition;

// 画像データをロードする関数（外部スレッドで実行）
void loadImageData(const std::string& folderPath) {
    // ここで実際に画像データを読み込む処理を行います
    // 仮にダミーデータを作成
    for (int i = 0; i < 10; ++i) {
        ImageData imgData;
        imgData.width = 128;
        imgData.height = 128;
        imgData.data = new unsigned char[128 * 128 * 4]; // RGBA

        // キューに追加
        {
            std::lock_guard<std::mutex> lock(queueMutex);
            imageQueue.push(imgData);
        }
        queueCondition.notify_one();
    }
}

// メインスレッドで実行する描画関数
void Render2D() {
    while (true) {
        ImageData imgData;
        {
            std::unique_lock<std::mutex> lock(queueMutex);
            queueCondition.wait(lock, [] { return !imageQueue.empty(); });
            imgData = imageQueue.front();
            imageQueue.pop();
        }

        // OpenGLコンテキストが有効なスレッドでglTexImage2Dを呼び出す
        glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, imgData.width, imgData.height, 0, GL_RGBA, GL_UNSIGNED_BYTE, imgData.data);

        // 画像データの解放
        delete[] imgData.data;

        // レンダリング処理をここに追加
        // ...
    }
}

int main() {
    // OpenGLコンテキストの初期化
    if (!glfwInit()) {
        std::cerr << "Failed to initialize GLFW" << std::endl;
        return -1;
    }

    GLFWwindow* window = glfwCreateWindow(800, 600, "OpenGL Example", nullptr, nullptr);
    if (!window) {
        std::cerr << "Failed to create GLFW window" << std::endl;
        glfwTerminate();
        return -1;
    }

    glfwMakeContextCurrent(window);
    glewExperimental = true;
    if (glewInit() != GLEW_OK) {
        std::cerr << "Failed to initialize GLEW" << std::endl;
        return -1;
    }

    // 画像読み込みスレッドを起動
    std::thread imageLoader(loadImageData, "image_path/folder_name");

    // メインループ
    while (!glfwWindowShouldClose(window)) {
        glClear(GL_COLOR_BUFFER_BIT);

        Render2D();

        glfwSwapBuffers(window);
        glfwPollEvents();
    }

    imageLoader.join();
    glfwDestroyWindow(window);
    glfwTerminate();

    return 0;
}
